<a href="https://colab.research.google.com/github/ehho34/PyTorch/blob/main/03_%EC%84%A0%ED%98%95%ED%9A%8C%EA%B7%80(Linear_Regression).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**03.선형회귀(Linear Regression)**

##**01-선형 회귀**

###**데이터에 대한 이해(Data Definition)**

In [ ]:
"""
-예측을 위해 사용하는 데이터 == 훈련 데이터셋(training dataset)
-학습이 끝난 후에 이 모델이 얼마나 잘 작동하는지 판별 == 테스트 데이터셋(test dataset)

-훈련 데이터셋의 구성
  -모델을 학습시키기 위한 데이터는 파이토치의 텐서 형태여야 함!
  -입력과 출력은 서로 다른 텐서에 저장해야 함!(입력은 x, 출력은 y)<-보편적으로~
  EX) 공부시간과 점수(아래 사진 참고)
  x_train = torch.FloatTensor([[1],[2],[3]])
  y_train = torch.FloatTensor([[2],[4],[6]])

-가설(Hypothesis) 수립
  -머신 러닝에서 식을 세울 때 이 식을 가설이라고 함

-선형 회귀
 -이 가설은 널리 알려져 있으니까 고민 불필요
 -선형 회귀란? : 학습 데이터와 가장 잘 맞는 하나의 직선을 찾는 일
                 일반적으로 이런 식     y = Wx + b 
                                     H(x) = Wx + b 을 가진다 (가설의 H를 따서 y대신 쓴 것뿐임)
                                     -여기서 x와 곱해지는 W를 가중치(Weight)라고 하며, b를 편향(bias)라고 함
                                     -W와 b는 중학교 수학 과정 직선의 방정식에서 기울기와 y절편에 해당
"""

In [ ]:
"""
-비용 함수(Cost Function)에 대한 이해
=손실 함수(loss function)=오차 함수(error function)=목적 함수(objective function)
★ 특히 비용 함수, 손실 함수 기억해두기 ★

-(실제값-예측값)의 제곱을 다 더하고 데이터의 개수로 나눈 것! -> 평균 제곱 오차(Mean Squared Error, MSE)
-Cost(W,b)를 최소로 만드는 W와 b를 구하면 훈련 데이터를 가장 잘 나타내는 직선을 구할 수 있다!
"""

In [ ]:
"""
-옵티마이저 - 경사 하강법(Gradient Descent)
-기울기가 지나티데 크면 실제값과 예측값의 오차가 커기고 기울기가 지나치게 작아도 마찬가지
-사실 b 또한 위와 같음

-cost가 최소화 되는 지점은 접선의 기울기가 0이 되는 지점 = 미분값이 0이 되는 지점
"""

###**파이토치로 선형 회구 구현하기**

####**1.기본 셋팅**

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
#현재 실습하고 있는 코드를 다시 실행해도 다음에도 같은 결과가 나올 수 있게 랜덤 시들들 줄이기
torch.manual_seed(1)

####**2.변수 선언**

In [4]:
x_train = torch.FloatTensor([[1],[2],[3]])
y_train = torch.FloatTensor([[2],[4],[6]])

In [7]:
print(x_train)
print(x_train.shape)

tensor([[1.],
        [2.],
        [3.]])
torch.Size([3, 1])


In [8]:
print(y_train)
print(y_train.shape)

tensor([[2.],
        [4.],
        [6.]])
torch.Size([3, 1])


####**3.가중치와 편향의 기초화**

In [9]:
#가중치 W를 0으로 초기화하고 학습을 통해 값이 변경되는 변수임을 명시함.
W = torch.zeros(1, requires_grad=True)  #requires_grad=True <- '이 변수는 학습을 통해 계속 값이 변경되는 변수임'
#가중치 W를 출력
print(W)

tensor([0.], requires_grad=True)


In [10]:
#편향 b를 0으로 초기화하고 얘도 학습을 통해 값이 변경되는 변수임을 명시하자!
b = torch.zeros(1, requires_grad=True)
print(b)

tensor([0.], requires_grad=True)


In [ ]:
#W와 b의 값이 모두 0이기 때문에 현재 이 직선의 방정식은 y = 0 X x + 0 --> x에 어떤 수가 들어가더라도 가설은 0을 예측할 것임!

####**4.가설 세우기**
H(x) = Wx + b

In [12]:
hypothesis = x_train * W + b
print(hypothesis)

tensor([[0.],
        [0.],
        [0.]], grad_fn=<AddBackward0>)


####**4.비용 함수 선언하기**

In [13]:
#평균 구하기(torch.mean 사용)
cost = torch.mean((hypothesis - y_train) ** 2)
print(cost)

tensor(18.6667, grad_fn=<MeanBackward0>)


####**6.경사 하강법 구현하기**ㅡ


In [14]:
optimizer = optim.SGD([W,b],lr=0.01)  #SDG(경사 하강법의 일종), lr(learning rate, 학습률)